In [2]:
from datasets import load_dataset

# load iris
iris = load_dataset("scikit-learn/iris")


Generating train split: 0 examples [00:00, ? examples/s]

In [35]:

from jax.numpy import vstack, array


iris_train = iris['train']

iris_train.set_format('jax', columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'])
X = vstack((iris_train['SepalLengthCm'], iris_train['SepalWidthCm'], iris_train['PetalLengthCm'], iris_train['PetalWidthCm']))
y_str = iris_train['Species']
# y = y_str.map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
# y_str is a list
y = array([0 if s == 'Iris-setosa' else 1 if s == 'Iris-versicolor' else 2 for s in y_str])
y_1hot = array([[1, 0, 0] if s == 'Iris-setosa' else [0, 1, 0] if s == 'Iris-versicolor' else [0, 0, 1] for s in y_str])
y_1hot.shape



(150, 3)

In [37]:
from ..berries.positional_encoding import get_positional_encoding
get_positional_encoding(4, 10)

ImportError: attempted relative import with no known parent package